# Лабораторная работа 4. Алгоритмы локального поиска.

### Выполнили: Баринов Даниил, Бугров Лев, Голованов Денис

In [1]:
import os
from tabulate import tabulate
import random
import numpy as np
import time

## Local search. 2 opt first-improvement + don't look bits

In [2]:
def cost(solution, dist, flow, size):
    cost_value = 0
    
    for i in range(size):
        for j in range(0, i):
                cost_value += dist[i][j] * flow[solution[i]][solution[j]]
    return cost_value*2

def random_heuristic(solution):
    random.shuffle(solution)
    

def swap(arr, i, j):
    arr[i],arr[j] = arr[j],arr[i]

In [3]:
def local_search_first_improvement(solution, dist, flow, size):
    global dont_look
    global solutions
    global best_cost
    global best_solution

    for i in range(size):
        for j in range(size):
            if j == i or dont_look[solution[j]] == 1 or dont_look[solution[i]] == 1:
                continue
            swap(solution, i, j)
            curr_cost = cost(solution, dist, flow, size)
            #print('Current solution:', curr_cost, solution)
            if best_cost > curr_cost:
                best_cost = curr_cost
                best_solution = solution
                solutions.append(solution.copy())
                return
            swap(solution, i, j)
        dont_look[solution[i]] = 1
        #print('Dont look:', dont_look)
        

def run_local_search_first_improvement(dist, flow, size):
    global dont_look
    global solutions
    global best_cost
    global best_solution
    
    dont_look = [0]*size
    solutions = []
    solution = [i for i in range(size)]
    random_heuristic(solution)
    best_solution = solution.copy()
    best_cost = cost(solution, dist, flow, size)
    #print('Initial solution:', best_cost, solution)
    solutions.append(solution)
    while len(solutions) > 0:
        local_search_first_improvement(solutions.pop(0), dist,flow,size)
    return best_cost, best_solution

## Iterated local search

In [4]:
def perturbation(solution):
    solution = list(solution)
    #print(solution)
    a = random.randint(0, len(solution))
    b = random.randint(0, len(solution))
    
    if a > b:
        a,b = b,a
        
    return list(solution[:a] + list(reversed(solution[a:b])) + solution[b::])

In [5]:
def common_local_search(solution, dist, flow, size):
    best_cost = cost(solution, dist, flow, size)
    best_solution = solution.copy()
    for i in range(size):
        for j in range(size):
            if j == i:
                continue
            swap(solution, i, j)
            curr_cost = cost(solution, dist, flow, size)
            if best_cost > curr_cost:
                solution_is_better = True
                best_cost = curr_cost
                best_solution = solution.copy()
                return best_cost, best_solution
            swap(solution, i, j)
    return best_cost, best_solution

In [6]:
def iterated_local_search(dist, flow, size, n_iter, n_repeats):
    solution = [i for i in range(size)]
    random_heuristic(solution)
    cost_value, solution = common_local_search(solution, dist, flow, size)
    best_cost = cost_value
    best_solution = solution.copy()
    for i in range(n_iter):
        solution = perturbation(solution)
        for j in range(n_repeats):
            cost_value, solution = common_local_search(solution, dist, flow, size)
        if best_cost > cost_value:
            best_cost = cost_value
            best_solution = solution.copy()
    return best_cost, best_solution

## Проверки на бенчмарках

In [7]:
base_dir = 'lab4_data'
benchmarks = os.listdir(base_dir)
data = dict.fromkeys(benchmarks)

for filename in benchmarks:
    with open(base_dir + '/' + filename) as f:
        lines = f.read().splitlines()
        N = int(lines[0].strip())
        D = []
        F = []

        for i in range(1, N + 1):
            D.append(list(map(int, lines[i].replace('  ', ' ').strip().split(' '))))

        for i in range(N + 2, 2*N + 2):
            F.append(list(map(int, lines[i].replace('  ', ' ').strip().split(' '))))

        data[filename] = {'size': N, 'dist':D, 'flow':F}

In [8]:
it_ls_results = dict.fromkeys(list(data.keys()))
for filename in data:
    print('Benchmark:', filename)
    dist = data[filename]['dist']
    flow = data[filename]['flow']
    size = data[filename]['size']
    best_res = [2**64, []]
    start_time = time.time()
    for i in range(10):
        res =  iterated_local_search(dist, flow, size, 100, 15)
        if res[0] < best_res[0]:
            best_res[0] = res[0]
            best_res[1] = res[1]
    end_time = time.time()
    it_ls_results[filename] = best_res + [round((end_time - start_time) / 10, 3)]
    print('Cost value:', it_ls_results[filename][0], '. Time:', it_ls_results[filename][2],'. Solution', it_ls_results[filename][1])

Benchmark: tai100a
Cost value: 23028132 . Time: 14.245 . Solution [75, 97, 4, 24, 98, 57, 10, 64, 90, 19, 77, 39, 95, 74, 6, 22, 93, 9, 63, 21, 61, 46, 11, 58, 69, 76, 15, 53, 81, 33, 17, 48, 51, 78, 31, 88, 3, 86, 7, 54, 92, 70, 50, 40, 23, 99, 2, 41, 34, 44, 85, 52, 55, 29, 20, 27, 62, 13, 30, 43, 87, 91, 18, 66, 94, 80, 84, 96, 60, 42, 68, 38, 0, 73, 32, 82, 1, 71, 59, 36, 89, 25, 35, 56, 37, 49, 12, 83, 16, 45, 26, 28, 5, 8, 65, 79, 47, 67, 14, 72]
Benchmark: tai20a
Cost value: 719520 . Time: 4.169 . Solution [0, 8, 12, 15, 9, 16, 18, 13, 10, 7, 2, 1, 5, 19, 3, 6, 11, 4, 14, 17]
Benchmark: tai40a
Cost value: 3292472 . Time: 3.897 . Solution [10, 5, 36, 3, 8, 27, 1, 2, 26, 0, 4, 16, 17, 9, 29, 7, 13, 33, 21, 31, 22, 24, 35, 14, 30, 39, 18, 32, 28, 12, 38, 25, 23, 20, 37, 11, 34, 19, 6, 15]
Benchmark: tai60a
Cost value: 7749702 . Time: 4.591 . Solution [21, 28, 7, 43, 24, 14, 30, 31, 13, 22, 33, 41, 9, 11, 44, 40, 1, 42, 26, 39, 47, 50, 58, 53, 12, 46, 10, 51, 49, 5, 34, 48, 25, 55, 

In [9]:
ls_results = dict.fromkeys(list(data.keys()))
for filename in data:
    print('Benchmark:', filename)
    dist = data[filename]['dist']
    flow = data[filename]['flow']
    size = data[filename]['size']
    best_res = [2**64, []]
    start_time = time.time()
    for i in range(10):
        res =  run_local_search_first_improvement(dist,flow,size)
        if res[0] < best_res[0]:
            best_res[0] = res[0]
            best_res[1] = res[1]
    end_time = time.time()
    ls_results[filename] = best_res + [round((end_time - start_time) / 10, 3)]
    print('Cost value:', ls_results[filename][0], '. Time:', ls_results[filename][2],'. Solution', ls_results[filename][1])

Benchmark: tai100a
Cost value: 21939418 . Time: 7.45 . Solution [35, 48, 17, 4, 55, 52, 88, 33, 51, 40, 84, 0, 16, 41, 80, 89, 3, 56, 44, 14, 63, 97, 34, 53, 36, 77, 38, 39, 67, 61, 42, 28, 90, 22, 86, 29, 58, 47, 13, 78, 7, 11, 19, 50, 70, 6, 43, 64, 68, 15, 60, 45, 26, 87, 37, 46, 5, 93, 76, 10, 23, 32, 27, 21, 54, 74, 73, 57, 69, 62, 1, 99, 12, 98, 20, 95, 59, 65, 91, 75, 30, 49, 71, 31, 81, 85, 79, 96, 83, 9, 82, 72, 24, 25, 94, 8, 66, 2, 18, 92]
Benchmark: tai20a
Cost value: 755438 . Time: 0.013 . Solution [15, 6, 12, 5, 19, 11, 3, 1, 0, 18, 7, 17, 8, 16, 2, 9, 14, 10, 4, 13]
Benchmark: tai40a
Cost value: 3324410 . Time: 0.132 . Solution [19, 33, 35, 0, 38, 15, 14, 9, 17, 31, 8, 13, 20, 24, 4, 5, 1, 11, 23, 34, 6, 28, 10, 22, 37, 16, 3, 25, 39, 2, 12, 29, 18, 27, 30, 26, 21, 32, 36, 7]
Benchmark: tai60a
Cost value: 7599984 . Time: 0.727 . Solution [29, 37, 52, 44, 19, 18, 50, 26, 30, 32, 1, 10, 16, 13, 12, 55, 56, 25, 15, 34, 22, 6, 5, 51, 28, 46, 24, 53, 3, 11, 23, 48, 40, 35, 9,

In [10]:
table_time = [[' ', 'Local search', 'Iterated local search']]
    
for benchmark in ls_results:
    table_time.append([benchmark,f'{ls_results[benchmark][0]} / {ls_results[benchmark][2]}s',f'{it_ls_results[benchmark][0]} / {it_ls_results[benchmark][2]}s'])

print('Лучший результат и среднее время за 10 запусков алгоритмов.')
print(tabulate(table_time, tablefmt='fancy_grid'))

Лучший результат и среднее время за 10 запусков алгоритмов.
╒═════════╤═══════════════════╤═══════════════════════╕
│         │ Local search      │ Iterated local search │
├─────────┼───────────────────┼───────────────────────┤
│ tai100a │ 21939418 / 7.45s  │ 23028132 / 14.245s    │
├─────────┼───────────────────┼───────────────────────┤
│ tai20a  │ 755438 / 0.013s   │ 719520 / 4.169s       │
├─────────┼───────────────────┼───────────────────────┤
│ tai40a  │ 3324410 / 0.132s  │ 3292472 / 3.897s      │
├─────────┼───────────────────┼───────────────────────┤
│ tai60a  │ 7599984 / 0.727s  │ 7749702 / 4.591s      │
├─────────┼───────────────────┼───────────────────────┤
│ tai80a  │ 14108460 / 2.628s │ 14711742 / 10.587s    │
╘═════════╧═══════════════════╧═══════════════════════╛


In [12]:
for filename in ls_results:
    with open('lab4_answers' + '/' + filename + '.sol', 'a') as f:
        f.write(' '.join(list(map(str,ls_results[filename][1]))))